<a href="https://colab.research.google.com/github/rani-sikdar/GenAI-complete-course-codes/blob/main/2_Simple_genai_app_using_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Simple GENAI App using Langchain

In [ ]:
pip install langchain langchain-openai langchain_community faiss-cpu

In [ ]:
pip install beautifulsoup4

In [ ]:
import os

OPENAI_API_KEY= "sk-proj-***"
LANGCHAIN_API_KEY ="*****"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LANGCHAIN_PROJECT"

LANGCHAIN_API_KEY ="******"
LANGCHAIN_TRACING_V2 = "true"
LANGCHAIN_PROJECT = "LANGCHAIN_PROJECT"

In [ ]:
#@title Data Ingestion -- data scrping from website

from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/chatbot/")
loader

In [ ]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content='\n\n\n\n\nBuild a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build

In [ ]:
#@title Roadmap --> Data Loading --> docs --> divide text into chunks --> convert into vectors(vector embedding) --> Vector store db

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size =200, chunk_overlap= 50)
documents = text_splitter.split_documents(docs)
documents[0]

Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content='Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain')

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model ="text-embedding-3-small")

/tmp/ipython-input-14-212168762.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model ="text-embedding-3-small")


In [ ]:
from langchain.vectorstores import FAISS
vectorstore_db = FAISS.from_documents(documents, embeddings)
vectorstore_db

In [ ]:
#@title query from a vector store db

query ="Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history"

resposne = vectorstore_db.similarity_search(query)
resposne[0].page_content


'Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.'

In [ ]:
#@title Retrieval Chains , Document Chains

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# create llm model
llm = ChatOpenAI(api_key= OPENAI_API_KEY, model= "gpt-3.5-turbo")

# create prompt template
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the following context:
    <context>
    {context}
    </context>
    """
)

# create document chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the following context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7e5715814810>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7e5715814c90>, root_client=<openai.OpenAI object at 0x7e57157f2790>, root_async_client=<openai.AsyncOpenAI object at 0x7e5715814890>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_doc

In [ ]:
query = "what is a langgraph?"
from langchain_core.documents import Document

document_chain.invoke({
    "input": query,
    "context":  [Document(page_content ="LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns. Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.")]
})

'What benefit does LangGraph provide for chat applications that support multiple conversational turns?'

In [ ]:
#@title retriever

# input --> retriever --> act as an interface --> get response from vector store db

retriever = vectorstore_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e57151b8190>, search_kwargs={})

In [ ]:
#@title creating retriever chains

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain) # we are using document_chain beacause it will be giving us context information
retrieval_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e57151b8190>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the following context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
      

In [ ]:
#@ Get the response from LLM

response = retrieval_chain.invoke({"input": "Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. "})
response['answer']

'How can ChatPromptTemplate help in turning raw user information into a format that the LLM can work with?'

In [ ]:
response

{'input': 'Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. ',
 'context': [Document(id='40bf4fd2-4e9f-4c23-b161-fbd90dc1b300', metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content="Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a"),
  Document(id='63a9bed6-3afd-4a06-9705-60329c6f2491', metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'Th

In [ ]:
response['context']

[Document(id='40bf4fd2-4e9f-4c23-b161-fbd90dc1b300', metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content="Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a"),
 Document(id='63a9bed6-3afd-4a06-9705-60329c6f2491', metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content="we are passing to the LLM. Let'